### Introduction

It is fraud detection problems and found there is unbalanced classifiction dataset. To solve that,
1. we need to test with undersampling, oversampling, mixsampling and non sampling dataset.  
    There are total 6 datasets. 2 undersampling datasets, 2 oversampling datsets, 1 mix sampling and non sampling dataset.
2. Train in many models and stacking it. (There are 20 machine learning modles, 1 stacking model, 2 ANN models and 1 CNN model)
3. Try to build Deep learning model.   
   
For testing puprose, NearMiss undersampling Dataset is mainly used. But If you want to try other dataset and models, just uncomment. 

In [1]:
import warnings
warnings.filterwarnings('ignore')
from IPython.display import Image

#Libraries used
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
plt.style.use('fivethirtyeight')
sns.set_style("whitegrid")

from plotly import tools 
import plotly.tools as tls
import plotly.offline as py
import plotly_express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import missingno as msno
py.init_notebook_mode(connected=True)

import cufflinks as cf 
import collections
from scipy.stats import norm

# Imblearn
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss

import tensorflow as tf 
import keras
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_predict
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import  RadiusNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import NuSVC
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from catboost import Pool, CatBoostClassifier, cv
import lightgbm as lgb
import xgboost as xgb

from sklearn.impute import SimpleImputer
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score
from sklearn.metrics import auc,roc_auc_score,roc_curve,classification_report,precision_recall_curve
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA

import random


2023-04-17 17:18:38.044177: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-17 17:18:38.535615: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-17 17:18:38.537560: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-17 17:18:40.020755: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Data Import

In [ ]:
import os 
datapath = os.path.join("../input/creditcardfraud/")

In [ ]:
import pandas as pd 
creditcard_df = pd.read_csv(datapath + "creditcard.csv")

### Data Visualization

In [ ]:
creditcard_df.head(3)

In [ ]:
creditcard_df["Class"].value_counts()

In [ ]:
creditcard_df.shape, creditcard_df.info()

In [ ]:
creditcard_df[["Time","Amount","Class"]].describe()

In [ ]:
# checking for duplcate entries
creditcard_df.duplicated().sum()

In [ ]:
# remove duplicate entries
creditcard_df.drop_duplicates(keep=False,inplace=True)

In [ ]:
timedelta = pd.to_timedelta(creditcard_df['Time'], unit='s')
creditcard_df['Time_min'] = (timedelta.dt.components.minutes).astype(int)
creditcard_df['Time_hour'] = (timedelta.dt.components.hours).astype(int)

In [ ]:
creditcard_df.tail()

In [ ]:
fig = make_subplots(rows=1,cols=2,subplot_titles=['Distribution of Time', 'Distribution of Amount'])
fig.add_trace(go.Histogram(name='Time',histnorm='probability',x=creditcard_df.Time),1,1)
fig.add_trace(go.Histogram(name='Amount',x=creditcard_df.Amount),1,2)
fig.update_xaxes(mirror=True,linecolor='black',linewidth=2,row=1,col=1)
fig.update_xaxes(mirror=True,linecolor='black',linewidth=2,row=1,col=2)
fig.update_yaxes(mirror=True,linecolor='black',linewidth=2,row=1,col=1)
fig.update_yaxes(mirror=True,linecolor='black',linewidth=2,row=1,col=2)
fig.update_layout(template='seaborn',width=700,height=300,margin=dict(t=50,b=0,l=0,r=0))
fig.show()

In [ ]:
class_0 = creditcard_df.loc[creditcard_df['Class'] == 0]["Time"]
class_1 = creditcard_df.loc[creditcard_df['Class'] == 1]["Time"]

hist_data = [class_0, class_1]
group_labels = ['Not Fraud', 'Fraud']

fig = ff.create_distplot(hist_data, group_labels, show_hist=False, show_rug=False)
fig['layout'].update(title='Credit Card Transactions Time Density Plot', xaxis=dict(title='Time [s]'))
iplot(fig, filename='dist_only')

In [ ]:
f, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(12,4))

bins = 50

ax1.hist(creditcard_df.Time[creditcard_df.Class == 1], bins = bins)
ax1.set_title('Fraud')

ax2.hist(creditcard_df.Time[creditcard_df.Class == 0], bins = bins)
ax2.set_title('Non_Fraud')

plt.xlabel('Time (in Seconds)')
plt.ylabel('Number of Transactions')
plt.show()

In [ ]:
#Exploring the distribuition by Class types throught hours and minutes
plt.figure(figsize=(12,5))
sns.distplot(creditcard_df[creditcard_df['Class'] == 0]["Time_hour"], 
             color='b')
sns.distplot(creditcard_df[creditcard_df['Class'] == 1]["Time_hour"], 
             color='r')
plt.title('Fraud x Normal Transactions by Hours', fontsize=17)
plt.xlim([-1,25])
plt.show()

In [ ]:
#Exploring the distribuition by Class types throught hours and minutes
plt.figure(figsize=(12,5))
sns.distplot(creditcard_df[creditcard_df['Class'] == 0]["Time_min"], 
             color='b')
sns.distplot(creditcard_df[creditcard_df['Class'] == 1]["Time_min"], 
             color='r')
plt.title('Fraud x Normal Transactions by minutes', fontsize=17)
plt.xlim([-1,61])
plt.show()

In [ ]:
f, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(12,6))

ax1.scatter(creditcard_df.Time[creditcard_df.Class == 1], creditcard_df.Amount[creditcard_df.Class == 1])
ax1.set_title('Fraud')

ax2.scatter(creditcard_df.Time[creditcard_df.Class == 0], creditcard_df.Amount[creditcard_df.Class == 0])
ax2.set_title('Normal')

plt.xlabel('Time (in Seconds)')
plt.ylabel('Amount')
plt.show()

In [ ]:
#Looking the Amount and time distribuition of FRAUD transactions
ax = sns.lmplot(y="Amount", x="Time_min", fit_reg=False,aspect=1.8,
                data=creditcard_df, hue='Class')
plt.title("Amounts by Minutes of Frauds and Normal Transactions",fontsize=16)
plt.show()

In [ ]:
ax = sns.lmplot(y="Amount", x="Time_hour", fit_reg=False,aspect=1.8,
                data=creditcard_df, hue='Class')
plt.title("Amounts by Hour of Frauds and Normal Transactions", fontsize=16)

plt.show()

In [ ]:
import matplotlib.gridspec as gridspec
#Looking the V's features
columns = creditcard_df.iloc[:,1:29].columns

frauds = creditcard_df.Class == 1
normals = creditcard_df.Class == 0

grid = gridspec.GridSpec(14, 2)
plt.figure(figsize=(15,20*4))

for n, col in enumerate(creditcard_df[columns]):
    ax = plt.subplot(grid[n])
    sns.distplot(creditcard_df[col][frauds], bins = 50, color='g') #Will receive the "semi-salmon" violin
    sns.distplot(creditcard_df[col][normals], bins = 50, color='r') #Will receive the "ocean" color
    ax.set_ylabel('Density')
    ax.set_title(str(col))
    ax.set_xlabel('')
plt.show()

In [ ]:
import seaborn as sns

plt.figure(figsize=(10,10))
dataplot=sns.heatmap(creditcard_df.corr())
plt.show()

### Data is highly imbalanced .We will correct it by oversamplying .After correcting we will check the skewness of the data 

* Data prepartion (No sampling, Undersampling, Oversampling, mix)  
* The undersampling take total numnder of minority class. In this case 460.  
* The oversampling take total number of majority class. In this case 282,493 

#### Non sampling dataset

In [ ]:
X = creditcard_df.drop(['Class'], axis=1)
y = creditcard_df['Class']
y.value_counts().to_frame()

In [ ]:
# X.drop(['Time'], axis=1, inplace=True)
X.head()

In [ ]:
X.duplicated().sum()

In [ ]:
y.head()

In [ ]:
### Note that you can fit_transform the whole oversampled features(train+test) from begining
sc=StandardScaler()
scaled_X=sc.fit_transform(X)
scaled_X

In [ ]:
scaled_X.shape, y.shape

In [ ]:
scaled_X = pd.DataFrame(scaled_X, columns=X.columns)
scaled_X.head()

#### Undersampling

NearMiss

In [ ]:
from imblearn.under_sampling import NearMiss
# Create an instance of NearMiss
nm = NearMiss()

# Fit and apply NearMiss to downsample the majority class
nm_features, nm_labels = nm.fit_resample(scaled_X, y)

nm_features.shape, nm_labels.shape, nm_labels.value_counts()


RandomUnderSampler

In [ ]:
rus = RandomUnderSampler(random_state=42)
rs_features, rs_labels = rus.fit_resample(scaled_X,y)
rs_features.shape, rs_labels.shape, rs_labels.value_counts()


#### Oversampling

In [ ]:
from imblearn.pipeline import make_pipeline as make_pipeline_imb # To do our transformation in a unique time
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.metrics import classification_report_imbalanced

SMOTE

In [ ]:
smote=SMOTE()
sm_features,sm_labels=smote.fit_resample(scaled_X,y)
sm_features.shape,sm_labels.shape, sm_labels.value_counts()

Adasyn

In [ ]:
# from imblearn.over_sampling import ADASYN
adasyn = ADASYN()
ad_features, ad_labels = adasyn.fit_resample(scaled_X, y)
ad_features.shape, ad_labels.shape, ad_labels.value_counts()

#### Mix Sampling

In [ ]:
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
import random

# Perform random under-sampling on the majority class
rus = NearMiss()
X_under, y_under = rus.fit_resample(scaled_X, y)
print(X_under.shape, y_under.shape)

# Perform random over-sampling on the minority class
ros = SMOTE()
X_over, y_over = ros.fit_resample(scaled_X, y)
print(X_over.shape, y_over.shape)

# Combine the under-sampled majority class and over-sampled minority class
mix_features = np.vstack([X_under, X_over])
mix_labels = np.hstack([y_under, y_over])

mix_features.shape, mix_labels.shape


In [ ]:
mix_features = pd.DataFrame(mix_features, columns=X.columns)

In [ ]:
mix_labels = pd.DataFrame(mix_labels)
mix_labels.columns = ['Class']
mix_labels.head()

In [ ]:
mix_labels.value_counts()

Search for duplicated because model can overfit easily

In [ ]:
print(f" NearMiss Duplication : {nm_features.duplicated().sum()} \n \
      Rus Duplication : {rs_features.duplicated().sum()} \n \
      SMOTE Duplication : {sm_features.duplicated().sum()} \n \
      ADASYN Duplication : {ad_features.duplicated().sum()} \n \
      Non Duplication : {scaled_X.duplicated().sum()} \n \
      Mix Duplication : {mix_features.duplicated().sum()}")

In [ ]:
Mix_df =pd.concat([mix_features, mix_labels],ignore_index=False,axis=1,sort=False)
Mix_df

In [ ]:
Mix_df.shape

In [ ]:
# remove duplicate entries
Mix_df.drop_duplicates(keep=False,inplace=True)
Mix_df.shape , Mix_df.duplicated().sum()

In [ ]:
mix_features = Mix_df.drop(["Class"],axis=1)
mix_labels = Mix_df['Class']
mix_features.shape, mix_labels.shape, mix_labels.value_counts()

#### Check skewness

In [ ]:
def check_skewness(x):
    # this can check relation between each column
    skew_limit=0.75
    skew_value=creditcard_df[x.columns].skew()
    #print(skew_value)
    skew_col=skew_value[abs(skew_value)>skew_limit]
    cols=skew_col.index
    return cols

In [ ]:
skewed_col_sm = check_skewness(sm_features)
skewed_col_ad = check_skewness(ad_features)
skewed_col_nm = check_skewness(nm_features)
skewed_col_rs = check_skewness(rs_features)
skewed_col_mix = check_skewness(mix_features)
skewed_col_non = check_skewness(scaled_X)
print(f"SMOTE col : \n {skewed_col_sm} \n  \
      Adasyn col : \n {skewed_col_ad} \n  \
      NearMiss col : \n {skewed_col_nm} \n  \
      Random col : \n {skewed_col_rs} \n  \
      Mix col : \n {skewed_col_mix} \n \
      NOn col : \n {skewed_col_non}") 

In [ ]:
from sklearn.preprocessing import PowerTransformer
pt=PowerTransformer(standardize=False)
sm_features[skewed_col_sm]=pt.fit_transform(sm_features[skewed_col_sm])
ad_features[skewed_col_ad]=pt.fit_transform(ad_features[skewed_col_ad])
nm_features[skewed_col_nm]=pt.fit_transform(nm_features[skewed_col_nm])
rs_features[skewed_col_rs]=pt.fit_transform(rs_features[skewed_col_rs])
mix_features[skewed_col_mix]=pt.fit_transform(mix_features[skewed_col_mix])
scaled_X[skewed_col_non]=pt.fit_transform(scaled_X[skewed_col_non])

### Train_test_split

In [ ]:
# This is just showing how to train_test_split and add the whole muliple dataset to one dictory

from sklearn.model_selection import train_test_split
Meta_Dataset = {}

print("Splitting Normal : ")
train_n_features, \
test_n_features, \
train_n_labels, \
test_n_labels=train_test_split(scaled_X,y,test_size=0.2,random_state=1)
print(train_n_features.shape, train_n_labels.shape, test_n_features.shape, test_n_labels.shape)
Meta_Dataset["None"] = [train_n_features.to_numpy(), train_n_labels.to_numpy(), test_n_features.to_numpy(), test_n_labels.to_numpy()]

print("Splitting SMOTE : ")
train_sm_features, \
test_sm_features, \
train_sm_labels, \
test_sm_labels=train_test_split(sm_features,sm_labels,test_size=0.2,random_state=1)
print(train_sm_features.shape, train_sm_labels.shape, test_sm_features.shape, test_sm_labels.shape)
Meta_Dataset["SMOTE"] = [train_sm_features.to_numpy(), train_sm_labels.to_numpy(), test_sm_features.to_numpy(), test_sm_labels.to_numpy()]

print("Splitting ADASYN : ")
train_ad_features, \
test_ad_features, \
train_ad_labels, \
test_ad_labels=train_test_split(ad_features,ad_labels,test_size=0.2,random_state=1)
print(train_ad_features.shape, train_ad_labels.shape, test_ad_features.shape, test_ad_labels.shape)
Meta_Dataset["ADASYN"] = [train_ad_features.to_numpy(), train_ad_labels.to_numpy(), test_ad_features.to_numpy(), test_ad_labels.to_numpy()]

print("Splitting NEARMISS : ")
train_nm_features, \
test_nm_features, \
train_nm_labels, \
test_nm_labels=train_test_split(nm_features,nm_labels,test_size=0.2,random_state=1)
print(train_nm_features.shape, train_nm_labels.shape, test_nm_features.shape, test_nm_labels.shape)
Meta_Dataset["NEARMISS"] = [train_nm_features.to_numpy(), train_nm_labels.to_numpy(), test_nm_features.to_numpy(), test_nm_labels.to_numpy()]

print("Splitting Random_Sampling : ")
train_rs_features, \
test_rs_features, \
train_rs_labels, \
test_rs_labels=train_test_split(rs_features,rs_labels,test_size=0.2,random_state=1)
print(train_rs_features.shape, train_rs_labels.shape, test_rs_features.shape, test_rs_labels.shape)
Meta_Dataset["Random Sampling"] = [train_rs_features.to_numpy(), train_rs_labels.to_numpy(), test_rs_features.to_numpy(), test_rs_labels.to_numpy()]

print("Splitting MIX : ")
train_mix_features, \
test_mix_features, \
train_mix_labels, \
test_mix_labels=train_test_split(mix_features,mix_labels,test_size=0.2,random_state=1)
print(train_mix_features.shape, train_mix_labels.shape, test_mix_features.shape, test_mix_labels.shape)
Meta_Dataset["Mixing"] = [train_mix_features.to_numpy(), train_mix_labels.to_numpy(), test_mix_features.to_numpy(), test_mix_labels.to_numpy()]


In [ ]:
# Check Meta_Dataset
for Name, values in Meta_Dataset.items():
   print(Name) 
   print(values[0].shape, values[1].shape, values[2].shape, values[3].shape)

If you take all of the Dataset is will be too long to train and test. For testing purpose, NearMiss Undersampling data is used

In [ ]:
# Selection NearMiss Datasets
train_nm_features   = train_nm_features.to_numpy()
train_nm_labels     = train_nm_labels.to_numpy() 
test_nm_features    = test_nm_features.to_numpy()
test_nm_labels      = test_nm_labels.to_numpy()
print(train_nm_features.shape, train_nm_labels.shape, test_nm_features.shape, test_nm_labels.shape)

### Model Builing (25 model)

In [ ]:
#Building Model Dict
Models = {
    "Logistic Regression": LogisticRegression(),                    #
    "Support Vector Classifier": SVC(),                             # Ridge, SVC, LinearSVC, Passive_AC
    "Decision Tree": DecisionTreeClassifier(max_depth=6),           #
    "KNearest": KNeighborsClassifier(n_neighbors=5),                # doesn't have model.predict_proba so I left out.
    "GaussianNB" : GaussianNB(),                                    #
    "LDA" : LinearDiscriminantAnalysis(),                           # 
    "Ridge" : RidgeClassifier(),                                    #  
    "QDA" : QuadraticDiscriminantAnalysis(),                        #
    "Bagging" : BaggingClassifier(),                                #
    "MLP" : MLPClassifier(),                                        #
    "LSVC" : LinearSVC(),                                           #  
    "BernoulliNB" : BernoulliNB(),                                  #  
    "Passive_AC" : PassiveAggressiveClassifier(),                   # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #  
    "SGB"     : GradientBoostingClassifier(n_estimators=100, random_state=9),
    "Adaboost" : AdaBoostClassifier(n_estimators=100, random_state=9, algorithm='SAMME.R', learning_rate=0.8),
    "Extra_T" : ExtraTreesClassifier(n_estimators=100, max_features=3),
    "R_forest" : RandomForestClassifier(max_samples=0.9, n_estimators=100, max_features=3),
    "XGB" : xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05)}

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import matthews_corrcoef

In [ ]:
# This function to calculate all of the evaluation matrix

def calculate_socre(classifier, test_features, test_labels):

    predictions   = classifier.predict(test_features)
    base_score   = classifier.score(test_sm_features,test_sm_labels)
    accuracy     = accuracy_score(test_labels, predictions)
    acc_bal      = balanced_accuracy_score(test_labels, predictions)
    av_precision = average_precision_score(test_labels, predictions)
    recall       = recall_score(test_labels, predictions)#Set df_used to the fraudulent transactions' dataset.
    f1           = f1_score(test_labels, predictions)
    roc          = roc_auc_score(test_labels, classifier.predict_proba(test_features)[:, 1])
    mcc          = matthews_corrcoef(test_labels, predictions)
    score = {
        "base_score"     : round(base_score,3),
        "accuary"        : round(accuracy,3),
        "acc_bal"        : round(acc_bal,3),
        "av_precision"   : round(av_precision,3),
        "recall"         : round(recall,3),
        "f1"             : round(f1,3),
        "roc"            : round(roc,3),
        "mcc"            : round(mcc,3)   }
    return score

In [ ]:
# This function train all the Dataset and all models
def Train_all_Model():

    Dataset_scores = {}

    for Name, values in Meta_Dataset.items():
        print(f"The {Name} Dataset is using.........")
        i = 1
        Model_scores = {} #Model = Socre
        for Model_Name, classifier in Models.items():
            print(f"{i}. {Model_Name}")
            classifier.fit(values[0], values[1])
            score = calculate_socre(classifier=classifier,
                                    test_features=values[2],
                                    test_labels=values[3])
            i = i+1
            print(f"{score}")
            Model_scores[Model_Name] = score
        print("________________________________________")
        Dataset_scores[Name] = Model_scores
    return Dataset_scores

This may take nearly 2 Hr to run all the models.

In [ ]:
# Dataset_scores = Train_all_Model()

For testing purpose, NearMiss undersampling dataset is used, and calculate_score function is modifed by removeing roc

In [ ]:
def calculate_nm_score(classifier, test_features=test_nm_features, test_labels=test_nm_labels):

    predictions   = classifier.predict(test_features)
    base_score   = classifier.score(test_sm_features,test_sm_labels)
    accuracy     = accuracy_score(test_labels, predictions)
    acc_bal      = balanced_accuracy_score(test_labels, predictions)
    av_precision = average_precision_score(test_labels, predictions)
    recall       = recall_score(test_labels, predictions)#Set df_used to the fraudulent transactions' dataset.
    f1           = f1_score(test_labels, predictions)
    mcc          = matthews_corrcoef(test_labels, predictions)
    score = {
        "base_score"     : round(base_score,3),
        "accuary"        : round(accuracy,3),
        "acc_bal"        : round(acc_bal,3),
        "av_precision"   : round(av_precision,3),
        "recall"         : round(recall,3),
        "f1"             : round(f1,3),
        "mcc"            : round(mcc,3)   }
    return score

In [ ]:
# for short in time,and resources, NearMiss undersampling data is taken
def train_nm_dataset():
    i = 1
    Model_scores = {} #Model = Socre
    for Model_Name, classifier in Models.items():
        print(f"{i}. {Model_Name}")
        classifier.fit(train_nm_features, train_nm_labels)
        score = calculate_nm_score(classifier=classifier)
        i = i+1
        print(f"{score}")
        print("________________________________________")
        Model_scores[Model_Name] = score
    return Model_scores

This may take short because of NearMiss undersampling dataset, but data result are not good

In [ ]:
nm_Model_score = train_nm_dataset()

In [ ]:
# lgb

params = {
          'boosting_type': 'gbdt',
          'objective': 'binary',
          'metric':'auc',
          'learning_rate': 0.05,
          'num_leaves': 7,  # we should let it be smaller than 2^(max_depth)
          'max_depth': 4,  # -1 means no limit
          'min_child_samples': 100,  # Minimum number of data need in a child(min_data_in_leaf)
          'max_bin': 100,  # Number of bucketed bin for feature values
          'subsample': 0.9,  # Subsample ratio of the training instance.
          'subsample_freq': 1,  # frequence of subsample, <=0 means no enable
          'colsample_bytree': 0.7,  # Subsample ratio of columns when constructing each tree.
          'min_child_weight': 0,  # Minimum sum of instance weight(hessian) needed in a child(leaf)
          'min_split_gain': 0,  # lambda_l1, lambda_l2 and min_gain_to_split to regularization
          'nthread': 8,
          'verbose': 0,
          'scale_pos_weight':150, # because training data is extremely unbalanced 
         }


lgb_train = lgb.Dataset(train_nm_features, train_nm_labels,  params={'verbose': -1})
lgb_test = lgb.Dataset(train_nm_features, train_nm_labels,  params={'verbose': -1}, reference=lgb_train)

LGBM = lgb.train(params, lgb_train, valid_sets=[lgb_train, lgb_test], verbose_eval=False)
LGBM_pred=LGBM.predict(test_nm_features)
LGBM_pred = LGBM_pred.round()


In [ ]:
LGBM_f1_score = f1_score(y_true=test_nm_labels,
                      y_pred=LGBM_pred)
LGBM_f1_score

In [ ]:
Catboost=CatBoostClassifier(eval_metric='Accuracy',use_best_model=True,random_seed=42,)
Catboost.fit(train_nm_features,train_nm_labels,eval_set=(test_nm_features,test_nm_labels))

In [ ]:
cat_pred = Catboost.predict(test_nm_features)
cat_F1_score = f1_score(y_true = test_nm_labels,
                  y_pred = cat_pred)
cat_F1_score

### Stacking + SMOTE 
Because of Training Time, Datasets and Models are selected just to test the stacking
* Base Models = Random Forest, K Nearest Neighbors
* Stack Models = Logistic Regression
* Dataset = Oversampling Dataset with SMOTE

In [ ]:
from sklearn.ensemble import StackingClassifier

base_models=[('RF',RandomForestClassifier(max_samples=0.9,n_jobs=-1)),('knn',KNeighborsClassifier(n_neighbors=5,n_jobs=-1))]
meta_model = LogisticRegression(n_jobs=-1)
stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_model, passthrough=True, cv=3,n_jobs=-1)

This take 20 min to run on my laptop

In [ ]:
# stacking_model.fit(train_sm_features,train_sm_labels)
# acc=stacking_model.score(test_sm_features,test_sm_labels)

On oversampling(SMOTE) dataset, the stack model get 0(False Negative) which is very good. But on undersampling data. It seem little overfit.

In [ ]:
stacking_model.fit(train_nm_features,train_nm_labels)
acc=stacking_model.score(test_nm_features,test_nm_labels)

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred = stacking_model.predict(test_nm_features)
conf_matrix = confusion_matrix(test_nm_labels, y_pred)
sns.heatmap(conf_matrix, annot = True, fmt='g')

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_nm_labels, y_pred))

In [ ]:
stack_f1_score = f1_score(y_true=test_nm_labels,
                          y_pred=y_pred)
stack_f1_score

### Tensorflow

#### Base Model

In [ ]:
n_inputs = sm_features.shape[1]
x_inputs = nm_features.shape[1]
print(n_inputs, x_inputs)

tf.set_seed = 42

base_model = Sequential([
    Dense(n_inputs, input_shape=(n_inputs,), activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

base_model.summary()

base_model.compile(
    optimizer = Adam(learning_rate= 0.0001),
    loss= 'binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
import itertools

# Create a confusion matrix
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=14)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
undersample_model = base_model
undersample_model.summary()

In [ ]:
tf.set_seed = 42
undersample_history = undersample_model.fit(train_nm_features,
                        train_nm_labels,
                        batch_size= 25,
                        epochs=20,
                        validation_data=(test_nm_features,test_nm_labels))

In [ ]:
undersample_predictions = undersample_model.predict(test_nm_features, batch_size=200, verbose=0)
undersample_predictions.shape

By comparing f1_score :  undersampling_ANN to stack, 

In [ ]:
undersample_fraud_predictions=np.round(undersample_predictions)
undersample_f1 = f1_score(y_true=test_nm_labels,
                          y_pred=undersample_fraud_predictions)
undersample_f1

In [ ]:
from sklearn.metrics import confusion_matrix
undersample_cm = confusion_matrix(test_nm_labels, undersample_fraud_predictions)
actual_cm = confusion_matrix(test_nm_labels, test_nm_labels)
labels = ['No Fraud', 'Fraud']

fig = plt.figure(figsize=(16,8))

fig.add_subplot(221)
plot_confusion_matrix(undersample_cm, labels, title="Random UnderSample \n Confusion Matrix", cmap=plt.cm.Reds)

fig.add_subplot(222)
plot_confusion_matrix(actual_cm, labels, title="Confusion Matrix \n (with 100% accuracy)", cmap=plt.cm.Greens)

In [ ]:
oversample_model = base_model
oversample_model.summary()

This model takes 30 min on my laptop

In [ ]:
# oversample_history = oversample_model.fit(train_sm_features,
#                         train_sm_labels,
#                         batch_size= 25,
#                         epochs=20,
#                         validation_data=(test_sm_features,test_sm_labels))

In [ ]:
# oversample_predictions = oversample_model.predict(test_sm_features, batch_size=200, verbose=0)
# oversample_predictions.shape

In [ ]:
# oversample_fraud_predictions=np.round(oversample_predictions)
# f1_oversample = f1_score(y_true=test_sm_labels,
#                          y_pred=oversample_fraud_predictions)
# f1_oversample

In [ ]:
# oversample_smote = confusion_matrix(test_sm_labels, oversample_fraud_predictions)
# actual_cm = confusion_matrix(test_sm_labels, test_sm_labels)
# labels = ['No Fraud', 'Fraud']

# fig = plt.figure(figsize=(16,8))

# fig.add_subplot(221)
# plot_confusion_matrix(oversample_smote, labels, title="OverSample (SMOTE) \n Confusion Matrix", cmap=plt.cm.Oranges)

# fig.add_subplot(222)
# plot_confusion_matrix(actual_cm, labels, title="Confusion Matrix \n (with 100% accuracy)", cmap=plt.cm.Greens)

From this far, we have seem stacking models and oversampling(SMOTE) DNN model are so great.

#### CNN model

In [ ]:
train_sm_features = np.array(train_sm_features)
test_sm_features = np.array(test_sm_features)

In [ ]:
# Reshaping the input to 3D.
train_sm_features_cnn = train_sm_features.reshape(train_sm_features.shape[0],train_sm_features.shape[1],1)
test_sm_features_cnn = test_sm_features.reshape(test_sm_features.shape[0],test_sm_features.shape[1],1)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers


# Build the model
cnn_model = keras.Sequential([
        # keras.Input(shape=input_shape),
        layers.Conv1D(32, 2, activation="relu",input_shape=train_sm_features_cnn[0].shape),
        layers.Dropout(0.5),
        layers.Conv1D(64, 2, activation="relu"),
        layers.Dropout(0.5),
        layers.Conv1D(64, 2, activation="relu"),
        layers.Dropout(0.5),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(1, activation="sigmoid"),
    ])
cnn_model.summary()

cnn_model.compile(
    optimizer = Adam(learning_rate= 0.0001),
    loss= 'binary_crossentropy',
    metrics=['accuracy']
)

Running this model almosts take 1Hr on my laptop, if you like to run, just run on gpu

In [ ]:
# cnn_history = cnn_model.fit(train_sm_features_cnn,
#                         train_sm_labels,
#                         batch_size= 25,
#                         epochs=20,
#                         validation_data=(test_sm_features_cnn,test_sm_labels))

In [ ]:
def plotLearningCurve(history,epochs=20):
  epochRange = range(1,epochs+1)
  plt.plot(epochRange,history.history['accuracy'])
  plt.plot(epochRange,history.history['val_accuracy'])
  plt.title('Model Accuracy')
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.legend(['Train','Validation'],loc='upper left')
  plt.show()

  plt.plot(epochRange,history.history['loss'])
  plt.plot(epochRange,history.history['val_loss'])
  plt.title('Model Loss')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.legend(['Train','Validation'],loc='upper left')
  plt.show()

In [ ]:
plotLearningCurve(undersample_history)

In [ ]:
# plotLearningCurve(oversample_history)

In [ ]:
# plotLearningCurve(cnn_history)

### Hyper-parameter Tuing

GridSearchCV and RandomizeSearchCV  

In general, GridSearchCV is best suited for smaller hyperparameter spaces, where it is feasible to test all possible combinations. RandomizedSearchCV is best suited for larger hyperparameter spaces, where it is not feasible to test all possible combinations, and random sampling can efficiently search a wide range of hyperparameters.

It's important to note that both GridSearchCV and RandomizedSearchCV can be combined with other techniques, such as cross-validation and feature selection, to further improve the performance of machine learning models. The choice of hyperparameter tuning technique ultimately depends on the size and complexity of the hyperparameter space, as well as the available computational resources.

#### Test for hyperparameter tuning

In [ ]:
#Convert to arrays to feed to the classifications algorithms
classifiers = {
    "Logistic Regression": LogisticRegression(),
    "Support Vector Classifier": SVC(),
    "Decision Tree": DecisionTreeClassifier(),
    "KNearest": KNeighborsClassifier()
}

cross_validation also take alot of time (more than 30 min ) therefore I change from SMOTE dataset to NEARMISS dataset

In [ ]:
#Calculate the cross-validation score for each classifier
print('Cross-Validation Scores:-')
for key, classifier in classifiers.items():
    classifier.fit(train_nm_features, train_nm_labels)
    cv_score = cross_val_score(classifier, train_nm_features, train_nm_labels, cv=3)
    print('{}: {}'.format(key,round(cv_score.mean()*100.0, 2)))

In [ ]:
params = {
    "Logistic Regression": {"penalty": [ 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]},
    "Support Vector Classifier": {'C': [0.5, 0.7, 0.9, 1], 'kernel': ['rbf', 'poly', 'sigmoid', 'linear']},
    "Decision Tree": {"criterion": ["gini", "entropy"], "max_depth": list(range(2,4,1)), 
              "min_samples_leaf": list(range(5,7,1))},
    "KNearest": {"n_neighbors": list(range(2,5,1)), 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']}
}

GridsearchCV

In [ ]:
#Using Grid search for finding the most optimal hyperparameters
def gridsearch(classifier, params):
    grid_classifier = GridSearchCV(classifier, params)
    grid_classifier.fit(train_nm_features, train_nm_labels)
    best_classifier = grid_classifier.best_estimator_
    return best_classifier

In [ ]:
#Scores after applying Grid Search
print('Cross-Validation Scores after applying GridSearch:-')
for key, classifier in classifiers.items():
    best_classifier = gridsearch(classifier,params[key])
    cv_score = cross_val_score(best_classifier, train_nm_features, train_nm_labels, cv=3)
    print('{}: {}'.format(key,round(cv_score.mean()*100.0, 2)))

RandomizedSearchCV

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
# from sklearn.ensemble import RandomForestClassifier

# # Create an instance of RandomizedSearchCV
# random_search = RandomizedSearchCV(models,  n_iter=20, cv=5)

# # Fit RandomizedSearchCV to the dataset
# random_search.fit(data, target)

# # Print the best hyperparameters and the corresponding mean cross-validation score
# print("Best hyperparameters: ", random_search.best_params_)
# print("Best score: ", random_search.best_score_)


## <a id='learn' > Learning Curve</a>
A learning curve refers to a plot of the prediction accuracy/error vs. the training set size (i.e: how better does the model get at predicting the target as you the increase number of instances used to train it). Usually both the training and test/validation performance are plotted together so we can diagnose the **bias-variance tradeoff** (i.e determine if we benefit from adding more training data, and assess the model complexity by controlling regularization or number of features).
<img src='https://histalk2.com/wp-content/uploads/2018/12/image-30.png' height=400 width=200/>

Just use logistic regression and NearMiss Datasets

In [ ]:
# Let's Plot LogisticRegression Learning Curve
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import learning_curve

def plot_learning_curve(estimator1, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    f, ((ax1)) = plt.subplots(1, figsize=(20,14), sharey=True)
    if ylim is not None:
        plt.ylim(*ylim)
    # First Estimator
    train_sizes, train_scores, test_scores = learning_curve(
        estimator1, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    ax1.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="#ff9124")
    ax1.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="#2492ff")
    ax1.plot(train_sizes, train_scores_mean, 'o-', color="#ff9124",
             label="Training score")
    ax1.plot(train_sizes, test_scores_mean, 'o-', color="#2492ff",
             label="Cross-validation score")
    ax1.set_title("Logistic Regression Learning Curve", fontsize=14)
    ax1.set_xlabel('Training size (m)')
    ax1.set_ylabel('Score')
    ax1.grid(True)
    ax1.legend(loc="best")
    return plt

In [ ]:
# cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=42)
# plot_learning_curve(log_reg, knears_neighbors, svc, tree_clf, X_train, y_train, (0.87, 1.01), cv=cv, n_jobs=4)

In [ ]:
log_reg = LogisticRegression()
log_reg.fit(train_nm_features,train_nm_labels)

In [ ]:
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=42)
plot_learning_curve(log_reg, train_nm_features, train_nm_labels, (0.87, 1.01), cv=cv, n_jobs=4)

## <a id='roc'>AUC-ROC Curve</a>
AUC - ROC curve is a performance measurement for classification problem at various thresholds settings. ROC is a probability curve and AUC represents degree or measure of separability. It tells how much model is capable of distinguishing between classes. Higher the AUC, better the model is at predicting 0s as 0s and 1s as 1s. By analogy, Higher the AUC, better the model is at distinguishing between patients with disease and no disease.

The ROC curve is plotted with TPR against the FPR where TPR is on y-axis and FPR is on the x-axis.
<img src='https://miro.medium.com/max/722/1*pk05QGzoWhCgRiiFbz-oKQ.png'/>

In [ ]:
#Predicting proba
y_pred_prob = log_reg.predict_proba(train_nm_features)[:, 1]
# roc_auc_score(test_nm_labels, log_reg.predict_proba(test_nm_features)[:, 1])

# Generate precision recall curve values: precision, recall, thresholds
precision, recall, thresholds = precision_recall_curve(train_nm_labels, y_pred_prob)

# Plot ROC curve
plt.plot(precision, recall)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision Recall Curve')
plt.show()

<!-- #ROC Curve
'''log_pred = cross_val_predict(log_reg, X_train, y_train, cv=5)
svc_pred = cross_val_predict(svc, X_train, y_train, cv=5)
tree_pred = cross_val_predict(decision_tree, X_train, y_train, cv=5)
knear_pred = cross_val_predict(knearest, X_train, y_train, cv=5)'''

log_pred = log_reg.predict(X_test)
svc_pred = svc.predict(X_test)
tree_pred = decision_tree.predict(X_test)
knear_pred = knearest.predict(X_test)

log_fpr, log_tpr, log_threshold = roc_curve(y_test, log_pred)
svc_fpr, svc_tpr, svc_threshold = roc_curve(y_test, svc_pred)
tree_fpr, tree_tpr, tree_threshold = roc_curve(y_test, tree_pred)
knear_fpr, knear_tpr, knear_threshold = roc_curve(y_test, knear_pred)

fig = go.Figure()
fig.add_trace(go.Scatter(name='Logistic Regression Classifier Score: {:.4f}'.format(roc_auc_score(y_test, log_pred)),
                         x=log_fpr,y=log_tpr,mode='lines'))
fig.add_trace(go.Scatter(name='Support Vector Classifier Score: {:.4f}'.format(roc_auc_score(y_test, svc_pred)),
                         x=svc_fpr,y=svc_tpr,mode='lines'))
fig.add_trace(go.Scatter(name='Decision Tree Classifier Score: {:.4f}'.format(roc_auc_score(y_test, tree_pred)),
                         x=tree_fpr,y=tree_tpr,mode='lines'))
fig.add_trace(go.Scatter(name='K-Nearest Neighbors Classifier Score: {:.4f}'.format(roc_auc_score(y_test, knear_pred)),
                         x=knear_fpr,y=knear_tpr,mode='lines'))
fig.add_trace(go.Scatter(name='AUC-ROC=0.5',x=[0,1],y=[0,1],line=dict(dash='dot'),showlegend=False))
fig.update_layout(
    width=700,xaxis=dict(mirror=True,linewidth=2,linecolor='black'),
    yaxis=dict(mirror=True,linewidth=2,linecolor='black'),
    title='ROC Curve<br>(All Classifiers)',
    template='seaborn',
    legend=dict(
        x=0.46,
        y=0,
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        bgcolor="Lightgray",
        bordercolor="Black",
        borderwidth=2
    ),
     annotations=[
        dict(
            x=0.5,
            y=0.5,
            xref="x",
            yref="y",
            text="Minimum ROC Score of 50% <br> (This is the minimum score to get)",
            showarrow=True,
            arrowhead=7,
            ax=40,
            ay=50
        )
    ]
)
fig.show() -->

### Final Socre on NearMiss Undersampling

In [ ]:
nm_score_df = pd.DataFrame(nm_Model_score)
nm_score_df = nm_score_df.transpose()
nm_score_df

In [ ]:
nm_f1_score_df = nm_score_df.f1
nm_f1_score_df

In [ ]:
new_df = {"LGBM_f1_score" : LGBM_f1_score, 
          "cat_F1_score" : cat_F1_score,
          "stack_f1_score" : stack_f1_score,
          "undersample_f1" : undersample_f1
}
new_df = pd.Series(new_df)
new_df

In [ ]:
nm_f1_score_df = nm_f1_score_df.append(new_df)

In [ ]:
nm_f1_score_df.columns = ["Models", "f1_score"]
nm_f1_score_df = pd.DataFrame(nm_f1_score_df, columns=[ "f1_score"])
nm_f1_score_df

In [ ]:
# sns.lineplot(x="Models",y="f1_socre", data=nm_f1_score_df)

### Good result output on oversampling with SMOTE dataset
They are just copy from above cells. 

In [ ]:
sm_stack = stacking_model

In [ ]:
sm_stack.fit(train_sm_features,train_sm_labels)
acc=sm_stack.score(test_sm_features,test_sm_labels)

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred_sm = sm_stack.predict(test_sm_features)
conf_matrix = confusion_matrix(test_sm_labels, y_pred_sm)
sns.heatmap(conf_matrix, annot = True, fmt='g')

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_sm_labels, y_pred_sm))

In [ ]:
stack_f1_score = f1_score(y_true=test_sm_labels,
                          y_pred=y_pred_sm)
stack_f1_score

In [ ]:
tf.set_seed = 125
oversample_history = oversample_model.fit(train_sm_features,
                        train_sm_labels,
                        batch_size = 25,
                        epochs = 20, # 20
                        validation_data = (test_sm_features,test_sm_labels))

In [ ]:
oversample_predictions = oversample_model.predict(test_sm_features, batch_size=200, verbose=0)
oversample_predictions.shape

In [ ]:
oversample_fraud_predictions=np.round(oversample_predictions)
f1_oversample = f1_score(y_true=test_sm_labels,
                         y_pred=oversample_fraud_predictions)
f1_oversample

In [ ]:
oversample_smote = confusion_matrix(test_sm_labels, oversample_fraud_predictions)
actual_cm = confusion_matrix(test_sm_labels, test_sm_labels)
labels = ['No Fraud', 'Fraud']

fig = plt.figure(figsize=(16,8))

fig.add_subplot(221)
plot_confusion_matrix(oversample_smote, labels, title="OverSample (SMOTE) \n Confusion Matrix", cmap=plt.cm.Oranges)

fig.add_subplot(222)
plot_confusion_matrix(actual_cm, labels, title="Confusion Matrix \n (with 100% accuracy)", cmap=plt.cm.Greens)

In [ ]:
plotLearningCurve(oversample_history)